# Zainab Aslam | 21I-2575 | BSCS-H | Assignment-2

**Question 1**

In [9]:
import numpy as np
import random

# Gernerating a 9*9 grid and populating 75% of grid randomly
def SudokuBoard():
    SudokoGrid = [[0] * 9 for _ in range(9)]
    for row in range(9):
        for col in range(9):
          # to check if randomly generated probability is less than .75
            if random.random() < 0.75:
                continue
            numbers = list(range(1, 10))

            random.shuffle(numbers)
            for num in numbers:
              # condition to check if the number is already present in the current row/col/3*3 grid
                if num not in SudokoGrid[row] and num not in [SudokoGrid[i][col] for i in range(9)]:
                    start_row = (row // 3) * 3
                    start_col = (col // 3) * 3
                    if num not in [SudokoGrid[start_row + i][start_col + j] for i in range(3) for j in range(3)]:
                        SudokoGrid[row][col] = num
                        break
    return SudokoGrid

# to check if a number can be placed in the specific position of grid
def ValidMove(row, column, number, SudokoGrid):
    for i in range(9):
        if SudokoGrid[row][i] == number or SudokoGrid[i][column] == number:
            return False
    startRow, startCol = 3 * (row // 3), 3 * (column // 3)
    # check for sub matrix of 3*3
    for i in range(3):
        for j in range(3):
            if SudokoGrid[startRow + i][startCol + j] == number:
                return False
    return True

# to check if the cell is empty 0 - indicated empty cell...
def EmptyCells(SudokoGrid):
    return sum(1 for row in SudokoGrid for cell in row if cell == 0)

# if the cell is empty here we get the position of the cell
def EmptyCellPosition(SudokoGrid):
    return [(i, j) for i in range(9) for j in range(9) if SudokoGrid[i][j] == 0]

# counting number of constraints each cell imposes on other empty cells
def LCV(row, col, SudokoGrid):
    # fetch empty cell position from the grid
    emptyCells = EmptyCellPosition(SudokoGrid)
    count = 0 # no of constraints
    for num in PossibleValues(row, col, SudokoGrid):
        for cell in emptyCells:
          # if the move is valid or not
            if ValidMove(cell[0], cell[1], num, SudokoGrid):
                count += 1
    return count

# degree: no of empty cells in the same row, col, and 3*3 grid
def Degree(row, col, SudokoGrid):
    count = 0
    for i in range(9):
      # check for row
        if SudokoGrid[row][i] == 0:
            count += 1
      # check for col
        if SudokoGrid[i][col] == 0:
            count += 1
    startRow, startCol = 3 * (row // 3), 3 * (col // 3)
    for i in range(3):
        for j in range(3):
          # check for 3*3 grid
            if SudokoGrid[startRow + i][startCol + j] == 0:
                count += 1
    return count

#function selects the next empty cell, using heuristic approach considering (for each cell)
# the number of possible values
# the degree
# the Least Constraining Value heuristic.

def NextEmptyCell(SudokoGrid):
    emptyCells = EmptyCellPosition(SudokoGrid)
    selectedCells = min(emptyCells, key=lambda cell: (
        len(PossibleValues(*cell, SudokoGrid)),
        -Degree(*cell, SudokoGrid),
        LCV(*cell, SudokoGrid)
    ))
    return selectedCells

# to check which possible value can be inserted in the cell, the value must satisfy sudoko rules
def PossibleValues(row, col, SudokoGrid):
    possibleVal = set(range(1, 10))
    for i in range(9):
        possibleVal.discard(SudokoGrid[row][i])
        possibleVal.discard(SudokoGrid[i][col])
    startRow, startCol = 3 * (row // 3), 3 * (col // 3)
    for i in range(3):
        for j in range(3):
            possibleVal.discard(SudokoGrid[startRow + i][startCol + j])
    return possibleVal

# checking consistency on the sudoko grid for each row / col / subgrid (3*3) for a given cell
def ArcConsistency(row, col, SudokoGrid):
    queue = [(row, col)]
    while queue:
        current_row, current_col = queue.pop(0)
        for i in range(9):
            if i != current_row and ValidMove(i, current_col, SudokoGrid[i][current_col], SudokoGrid):
                if SudokoGrid[i][current_col] == 0:
                    return False
                if SudokoGrid[i][current_col] in PossibleValues(current_row, current_col, SudokoGrid):
                    SudokoGrid[current_row][current_col] = SudokoGrid[i][current_col]
                    queue.append((current_row, current_col))
                    break
        for j in range(9):
            if j != current_col and ValidMove(current_row, j, SudokoGrid[current_row][j], SudokoGrid):
                if SudokoGrid[current_row][j] == 0:
                    return False
                if SudokoGrid[current_row][j] in PossibleValues(current_row, current_col, SudokoGrid):
                    SudokoGrid[current_row][current_col] = SudokoGrid[current_row][j]
                    queue.append((current_row, current_col))
                    break
        startRow, startCol = 3 * (current_row // 3), 3 * (current_col // 3)
        for i in range(startRow, startRow + 3):
            for j in range(startCol, startCol + 3):
                if (i != current_row or j != current_col) and ValidMove(i, j, SudokoGrid[i][j], SudokoGrid):
                    if SudokoGrid[i][j] == 0:
                        return False
                    if SudokoGrid[i][j] in PossibleValues(current_row, current_col, SudokoGrid):
                        SudokoGrid[current_row][current_col] = SudokoGrid[i][j]
                        queue.append((current_row, current_col))
                        break
    return True

# The function iterates through each empty cell in the grid,
# selects a cell to fill based on heuristics
# calculates possible values for that cell

def SudokuSolution(SudokoGrid):
    emptyCells = EmptyCellPosition(SudokoGrid)
    if not emptyCells:
        return SudokoGrid
    cell = NextEmptyCell(SudokoGrid)
    row, col = cell
    possibleVal = PossibleValues(row, col, SudokoGrid)
    # back tracking
    for num in possibleVal:
        if ValidMove(row, col, num, SudokoGrid):
            SudokoGrid[row][col] = num
            if ArcConsistency(row, col, SudokoGrid):
                solution = SudokuSolution(SudokoGrid)
                if solution:
                    return solution
            SudokoGrid[row][col] = 0
    return None

def PrintSol(SudokoGrid):
    if SudokoGrid:
        print("\nSolved Sudoku Board:")
        print(np.matrix(SudokoGrid))
        print("\nSolution Exists")
    else:
        print("Solution does not Exist.")

# Entry Point of the code
NewGrid = SudokuBoard()
print("Randomly Generated Sudoku Board:")
print(np.matrix(NewGrid))
PrintSol(SudokuSolution(NewGrid))


Randomly Generated Sudoku Board:
[[0 0 0 0 0 2 0 0 3]
 [0 0 0 0 0 4 0 0 0]
 [0 0 0 0 0 0 0 0 4]
 [0 0 0 4 0 9 2 0 0]
 [0 0 0 0 0 0 0 0 0]
 [6 0 0 0 0 0 5 0 0]
 [0 4 0 1 0 0 8 0 6]
 [0 8 3 0 6 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]]

Solved Sudoku Board:
[[4 1 6 5 8 2 9 7 3]
 [7 2 9 3 1 4 6 8 5]
 [3 5 8 7 9 6 1 2 4]
 [8 3 1 4 5 9 2 6 7]
 [5 7 2 6 3 8 4 1 9]
 [6 9 4 2 7 1 5 3 8]
 [9 4 7 1 2 3 8 5 6]
 [2 8 3 9 6 5 7 4 1]
 [1 6 5 8 4 7 3 9 2]]

Solution Exists


**Question 2**

In [10]:
import numpy as np
import random

class GeneticAlgo_MagicSquare:
  # constructor to initialize all the attr
    def __init__(self, size=3, PopulationSize=100, MutationRate=0.1):
        self.size = size
        self.PopulationSize = PopulationSize
        self.MutationRate = MutationRate
        self.TargetSum = size * (size ** 2 + 1) // 2 # 15
        self.population = self.InitializePopulation()
        self.BestSolution = None
  # matrix is generated for population
    def InitializePopulation(self):
        population = []
        while len(population) < self.PopulationSize:
            individual = self.RandomSquare()
            # check fitness
            if self.CalcFitness(individual):
              # if fitness is good enough include it in population
                population.append(individual)

        return population

  # a random square is generated that returns shuffled matrix
    def RandomSquare(self):
        Square = list(range(1, self.size ** 2 + 1))
        random.shuffle(Square)
        return Square
  # return true or false for fitness
    def CalcFitness(self, individual):
        matrix = np.array(individual).reshape(self.size, self.size)
        rowSum = np.sum(matrix, axis=1)
        colSum = np.sum(matrix, axis=0)
        mainDiagonalSum = np.trace(matrix)
        secDiagonalSum = np.trace(np.flip(matrix, axis=1))
        # if row col diagonals sums == target sum
        return (np.all(rowSum == self.TargetSum) and
                np.all(colSum == self.TargetSum) and
                mainDiagonalSum == self.TargetSum and
                secDiagonalSum == self.TargetSum)

    def UpdateFitness(self, individual, fitness):
        if fitness > 0:
          # compare the fitness the solution with the fitness of best solution
            if self.BestSolution is None or fitness > self.bestFit:
              # if fitness of this is better, select it as best
                self.BestSolution = individual
                self.bestFit = fitness

    def Mutation(self, individual):
      # keeping save the orignal indv
        MutateOne = individual[:]
        # if rand generated prob less than mutation rate
        if random.random() < self.MutationRate:
          # select random index for mutation
            idx1, idx2 = random.sample(range(len(individual)), 2)
            # performing mutation
            MutateOne[idx1], MutateOne[idx2] = MutateOne[idx2], MutateOne[idx1]
        return MutateOne

    def Evolution(self, max_generations=1000):
      # display generated matrix
        print("Randomly Generated Grid:")
        print(np.array(self.population[0]).reshape(self.size, self.size))
        print()

        for generation in range(max_generations):
            # for each gen there is population
            newPopulation = []
            for individual in self.population:
              # calc fitness of each population
                fitness = self.CalcFitness(individual)
                # if fitness is better replace it
                self.UpdateFitness(individual, fitness)
                if fitness:
                    break
                newPopulation.append(individual)
                # perform mutation
                newPopulation.append(self.Mutation(individual))
            if fitness:
                break
            self.population = newPopulation

        print("Evolution completed.")
        if self.BestSolution is not None:
            print("Best Solution:")
            print(np.array(self.BestSolution).reshape(self.size, self.size))
        else:
            print("No valid solution found.")

Square_solver = GeneticAlgo_MagicSquare()
Square_solver.Evolution()


Randomly Generated Grid:
[[6 1 8]
 [7 5 3]
 [2 9 4]]

Evolution completed.
Best Solution:
[[6 1 8]
 [7 5 3]
 [2 9 4]]
